In [86]:
import pandas as pd
import numpy as np
from pathlib import Path
from pandas import read_table
from pylab import rcParams
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.core.display import HTML 
from matplotlib.pyplot import figure
import warnings
import datetime
from scipy.stats import chi2_contingency
from pqdm.processes import pqdm
warnings.filterwarnings('ignore')

In [87]:
class Logbook(object):
    def __init__(self, name):
        self.name = name
        self.notes = []
        self.params = {}
        print(f'Logbook ~{name}~ created.')
    
    def log(self, note, category='misc'):
        self.notes.append((category, note))
        
    def retrieve(self, category=None):
        df = pd.DataFrame(self.notes, columns = ['category', 'note'])
        if category:
            return df[df.category==category]
        else:
            return df
        
    def log_param(self, name, param):
        self.params[name] = param
    
    def __len__(self):
        return len(self.notes)

In [88]:
logbook = Logbook('PerturbNet-HNSC')

Logbook ~PerturbNet-HNSC~ created.


## Goal: Learn a three-layer network from Z (traits), Y (gene expressions), and X (SNP data).

Z: array of shape (n, r)
            
    The data matrix with n samples and r traits. ==>> traits
        
Y: array of shape (n_o, q)
    
    The data matrix with n_o samples and q genes. ==>> expression

X: array of shape (n, p) 
    ==>> mutations 

_________

In [89]:
cbioportal_datahub = 'https://github.com/cBioPortal/datahub/tree/master/public'

In [90]:
special_genes = """
RELA
PBX1
SPI1
HIVEP1
MXI1
TBX19
DNMT1
SMARCC1
ZNF410
HMBOX1
TFEB
HINFP
NFYA
BPTF
CREB1
AR
STAT6
TERT
""".split()

In [91]:
# Gene Expression Differences Associated with Human Papillomavirus Status in Head and Neck Squamous Cell Carcinoma
# https://clincancerres.aacrjournals.org/content/12/3/701
data_tables = pd.read_html('https://clincancerres.aacrjournals.org/content/12/3/701.figures-only')
known_genes = data_tables[2]['HUGO ID'].dropna().values.reshape(-1, )

_____________

In [92]:
root_dir = Path('/ihome/hosmanbeyoglu/kor11/tools/PerturbNet/TCGA')
cancer = 'HNSC'
logbook.log_param('cancer', cancer)

In [93]:
expression_url = 'https://media.githubusercontent.com/media/cBioPortal/datahub/master/public/hnsc_tcga_pan_can_atlas_2018/data_mrna_seq_v2_rsem.txt'
phenotype_url = 'https://media.githubusercontent.com/media/cBioPortal/datahub/master/public/hnsc_tcga_pan_can_atlas_2018/data_clinical_patient.txt'
genotype_url = 'https://media.githubusercontent.com/media/cBioPortal/datahub/master/public/hnsc_tcga_pan_can_atlas_2018/data_mutations.txt'

In [94]:
rnaseq = pd.read_table(expression_url)
clinical = pd.read_table(phenotype_url)
mutations = pd.read_table(genotype_url)

In [95]:
clinical = pd.read_table(phenotype_url)
clinical = clinical.drop([0, 1, 2, 3])
clinical = clinical.set_index('#Patient Identifier')
traits = clinical[['Subtype']].copy()
traits.index.name = None
traits.columns = ['hpv']
traits = traits.dropna()
traits.hpv = traits.hpv.replace({'HNSC_HPV-': 0, 'HNSC_HPV+': 1})
traits

,hpv
TCGA-4P-AA8J,0
TCGA-BA-4074,0
TCGA-BA-4076,0
TCGA-BA-4078,0
TCGA-BA-5149,0
...,...
TCGA-UF-A7JT,0
TCGA-UF-A7JV,0
TCGA-UP-A6WW,1
TCGA-WA-A7GZ,0


In [96]:
m0 = len(mutations)
mutations = mutations[(mutations['Variant_Classification'] != 'Silent')]
m1 = len(mutations)
mutations = mutations[(mutations['IMPACT'] != 'LOW')]
m2 = len(mutations)
print(f'Filtered out {m0-m2} samples')
logbook.log(f'Filtered out {m0-m2} samples', 'filtering')

Filtered out 29288 samples


In [97]:
mutated_genes = pd.read_csv(root_dir / cancer / 'Mutated_Genes.txt', delimiter='\t')
mutated_genes['Freq'] = mutated_genes['Freq'].str[:-1].astype(float)

In [98]:
mutations['Barcode'] = mutations['Tumor_Sample_Barcode'].str[:-3]

In [99]:
mutation_matrix = pd.DataFrame(columns=set(mutations['Hugo_Symbol']), index = traits.index).fillna(0)

In [100]:
for patient_id in traits.index:
    for m in set(mutations[mutations.Barcode == patient_id]['Hugo_Symbol']):
        mutation_matrix.loc[patient_id, m] = 1

In [101]:
rnaseq.index = rnaseq['Hugo_Symbol']
rnaseq = rnaseq.drop(['Hugo_Symbol', 'Entrez_Gene_Id'], axis=1)
rnaseq.columns = rnaseq.columns.str[:-3]
rnaseq = rnaseq[rnaseq.index.notnull()].dropna().drop_duplicates()
rnaseq = rnaseq[~rnaseq.index.duplicated(keep='first')]
rnaseq = rnaseq.loc[:,~rnaseq.columns.duplicated()]
rnaseq.index.name = None

In [102]:
expression_cutoff = 100
rnaseq = rnaseq[rnaseq.mean(1) > expression_cutoff]
logbook.log_param('expression_cutoff', expression_cutoff)

In [103]:
mutation_matrix = mutation_matrix[set(mutated_genes.query('Freq > 5').sort_values(by='Freq', ascending=False).Gene) & set(mutations['Hugo_Symbol'])]

In [104]:
rnaseq = np.log2(rnaseq + 1)

In [105]:
rnaseq = rnaseq[~(rnaseq.var(1) < 0.1)]

In [106]:
from collections import namedtuple
from scipy.stats import ttest_ind as ttest

Result = namedtuple('Result', ['hpv_positive', 'hpv_negative', 'pvalue'])

def compare(gene, df=rnaseq):

    hpv_p = df.loc[gene][traits[traits.hpv == 1].index]
    hpv_n = df.loc[gene][traits[traits.hpv == 0].index]

    hpv_p_mean = hpv_p.mean()
    hpv_p_std = hpv_p.std()

    hpv_n_mean = hpv_n.mean()
    hpv_n_std = hpv_n.std()

    pvalue = ttest(hpv_p, hpv_n).pvalue

    # return Result([hpv_p_mean, hpv_p_std], [hpv_n_mean, hpv_n_std], pvalue)
    return pvalue


In [120]:
compare('NSD1')

1.3064058052599494e-09

In [107]:
pvals = pqdm(rnaseq.index, compare, 20)

SUBMITTING | :   0%|          | 0/12295 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/12295 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/12295 [00:00<?, ?it/s]

In [108]:
top_diff_expr_genes = 5000
rnaseq = rnaseq.loc[pd.DataFrame(pvals, index=rnaseq.index).sort_values(by=0)[:top_diff_expr_genes].index]
logbook.log_param('top_diff_expr_genes', top_diff_expr_genes)

In [109]:
common_samples = set(rnaseq.columns) & set(mutation_matrix.index)
rnaseq = rnaseq[common_samples]
mutation_matrix = mutation_matrix.loc[common_samples]

traits = traits.loc[set(mutation_matrix.index) & set(traits.index)].astype(int)
mutation_matrix = mutation_matrix.loc[set(mutation_matrix.index) & set(traits.index)].astype(int)
rnaseq = rnaseq[set(mutation_matrix.index) & set(traits.index)]

In [110]:
rnaseq = rnaseq.subtract(rnaseq.mean(1), 0).div(rnaseq.std(1), 0)

In [126]:
rnaseq.shape, traits.shape, mutation_matrix.shape

((5000, 487), (487, 1), (487, 116))

In [112]:
Z = traits.to_numpy()
Y = rnaseq.T.to_numpy()
X = mutation_matrix.to_numpy()

Z.shape, Y.shape, X.shape

((487, 1), (487, 5000), (487, 116))

In [113]:
r = Z.shape[1]
n = X.shape[0]
q = Y.shape[1]
p = X.shape[1]
assert X.shape[0]==Z.shape[0]
assert Y.shape[0]<=n

print(f'{r} traits\n{n} samples,\n{q} genes\n{p} mutations')

1 traits
487 samples,
5000 genes
116 mutations


In [114]:
logbook.log_param('matrices', {'traits': r, 'samples': n, 'genes': q, 'mutations': p})

In [115]:
root_dir / cancer

PosixPath('/ihome/hosmanbeyoglu/kor11/tools/PerturbNet/TCGA/HNSC')

In [116]:
np.savetxt(root_dir / cancer / 'traits.txt', Z, delimiter='\t', fmt='%s')
np.savetxt(root_dir / cancer / 'expression.txt', Y, delimiter='\t', fmt='%s')
np.savetxt(root_dir / cancer / 'genotype.txt', X, delimiter='\t', fmt='%s')
traits.to_csv(root_dir / cancer / 'traits_data.csv')
mutation_matrix.to_csv(root_dir / cancer / 'mutations_data.csv')
rnaseq.to_csv(root_dir / cancer / 'rnaseq_data.csv')
logbook.log_param('timestamp', datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

In [117]:
import json

with open(root_dir / cancer / 'nb_params.json', 'w') as f:
    f.write(json.dumps(logbook.params))
    
print(f'Saved params to {root_dir / cancer / "nb_params.json"}')

Saved params to /ihome/hosmanbeyoglu/kor11/tools/PerturbNet/TCGA/HNSC/nb_params.json
